In [1]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [2]:
data = pd.read_excel('/home/aleksandr/Документы/GitHub/data/ML6/data_ford_price.xlsx')

#  Отбор признаков: мотивация

Отбор признаков — это процесс выбора важных признаков, оказывающих наибольшее влияние на предсказание.

## Предобработка данных

In [7]:
# Давайте оценим влияние мультиколлинеарности на линейную регрессию:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

y = data['price']
x = data.drop(columns='price')

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

## Обучение модели

In [5]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4682.957


## Удаление избыточного признака

Мы выяснили, что у нас присутствует сильная зависимость между lat и weather. Удалим lat, так как этот признак, в отличие от weather, необходимо округлять.

In [6]:
x.drop('lat', axis = 1, inplace = True)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [8]:
model = LinearRegression()
model.fit(X_train, y_train)
y_predicted = model.predict(X_test)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 4672.930


#  Отбор признаков: классификация методов

Методы отбора признаков предназначены для уменьшения количества входных переменных до тех значений, которые наиболее полезны для предсказательной способности модели.

## Метод рекурсивного исключения признаков

Метод рекурсивного исключения признаков (RFE) предполагает выбор признаков путём рекурсивного рассмотрения всё меньших и меньших наборов фичей.

In [3]:
from sklearn.feature_selection import RFE

In [8]:
y = data['price']
x = data.drop(columns='price')

In [9]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [10]:
# Выделим три наиболее значимых признака:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [11]:
# узнаем, как RFE проранжировал все доступные признаки:
X_train.columns

Index(['year', 'cylinders', 'odometer', 'lat', 'long', 'weather'], dtype='object')

In [12]:
selector.ranking_

array([1, 1, 4, 1, 3, 2])

##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

<center> <img src = https://lms.skillfactory.ru/assets/courseware/v1/8f55a77ec813c98e6e48f2efff923f23/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-ml6-9_3.png alt="drawing" style="width:800px;">

Библиотека sklearn обеспечивает реализацию большинства полезных статистических показателей, например:
- коэффициента корреляции Пирсона: f_regression();
- дисперсионного анализа ANOVA: f_classif();
- хи-квадрата: chi2();
- взаимной информации: mutual_info_classif() и mutual_info_regression().

библиотека SciPy обеспечивает реализацию многих других статистических данных, таких как тау Кендалла (kendalltau) и ранговая корреляция Спирмена (spearmanr).

sklearn также предоставляет множество различных методов фильтрации после расчёта статистики для каждой входной переменной с целевой.
- выбор k лучших переменных: SelectKBest;
- выбор переменных верхнего процентиля: SelectPercentile.

In [13]:
from sklearn.feature_selection import SelectKBest, f_regression

In [14]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [ ]:
# Обучите модель линейной регрессии на найденных двумя способами трёх важных признаках и сравните полученные результаты.
# 1 балл	Верно выделены три столбца-признака для обучения, выбранные RFE.
# 1 балл	Верно выделены три столбца-признака для обучения, выбранные SelectKBest.
# 3 балла	Обучена регрессия на первых трёх столбцах, оценено качество модели на тесте.
# 3 балла	Обучена регрессия на вторых трёх столбцах, оценено качество модели на тесте.
# 2 балла	Произведено сравнение выбранных метрик в форме комментария (в текстовой ячейке).